In [57]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
n_inputs = 13
n_hidden = 20
n_classes = 3

num_samples = 133

In [59]:
def logits_function(p, X):
    # Roll-back the weights and biases
    W1 = p[0:260].reshape((n_inputs,n_hidden))
    b1 = p[260:280].reshape((n_hidden,))
    W2 = p[280:340].reshape((n_hidden,n_classes))
    b2 = p[340:343].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [60]:
# Forward propagation
def forward_prop(params, X,y):
    logits = logits_function(params, X)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [61]:
def Get_PSO(f):
    # Initialize swarm
    options = {'c1': 2.05, 'c2': 2.05, 'w':0.3}

    # Call instance of PSO
    dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
    optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=dimensions, options=options)

    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=1000)
    return pos

In [62]:
def predict(pos, X):
    logits = logits_function(pos, X)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [63]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

labelencoder = LabelEncoder()

def Start_Wine():
    data = pd.read_csv('databases/wine.data', names = ['class', 'alcohol', 'malic acid', 'ash', 'alcalinity of ash', 'magnesium', 'total phenols', 'flavanoids', 'nonflavanoid phenols', 'proanthocyanins', 'color intensity', 'hue', 'diluted', 'proline'])

    target = labelencoder.fit_transform(data['class'].values)
    data_drop = data.drop('class',axis=1)
    data = data_drop.values

    sc = StandardScaler()
    data = sc.fit_transform(data)

    x_train, x_test, y_train, y_test = train_test_split(data, target)

    def f(x):
        n_particles = x.shape[0]
        j = [forward_prop(x[i], x_train, y_train) for i in range(n_particles)]
        return np.array(j)

    pos = Get_PSO(f)
    scores = (predict(pos, x_test) == y_test)

    print("\n Wine \n")
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())

In [64]:
Start_Wine()

2022-03-25 13:29:29,311 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 2.05, 'c2': 2.05, 'w': 0.3}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=5.49e-6
2022-03-25 13:29:35,194 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 5.4891494961118885e-06, best pos: [ 3.17113146e+00  2.77229320e-01 -1.25017031e+00 -2.83363077e-01
  9.04721661e+00 -3.10692389e+00  3.90403370e-01 -1.24053368e+00
 -7.04837339e-01  6.59607940e-01  2.78961460e+00  3.88512915e+00
  1.68498364e+00  1.83108430e+00  1.08622001e-01 -1.38514529e-01
 -1.78191750e+00  1.61932393e+00  2.66773274e-01  1.27133196e+00
  1.30668397e+00  4.00892165e+00  1.67457515e+00  3.62165833e-01
  1.26449250e+00  4.43122718e-01  5.13441772e-01  8.96254074e-01
  1.49697241e+00  1.47409921e+00  9.84605646e-01  7.31863989e-01
  2.02467178e+00  7.23164734e-01  2.59959436e+00  7.35725016e-01
  7.07091658e-01 -4.53346157e-01 -4.99786439e-01  9.75455834e-01
 -4.50173624e-01 


 Wine 

Mean:  0.9777777777777777
Standard Deviation:  0.14740554623801777
